In [1]:
unique_plus_peaks=($(cat tts/*_enriched-LUZ7/*_enriched-LUZ7.plus.peaks.oracle.narrowPeak.counts.clustered.csv | sort -t ',' -k14 -n -u | awk -F ',' 'NR>1 {print $14}'))
unique_minus_peaks=($(cat tts/*_enriched-LUZ7/*_enriched-LUZ7.minus.peaks.oracle.narrowPeak.counts.clustered.csv | sort -t ',' -k14 -n -u | awk -F ',' 'NR>1 {print $14}'))

In [2]:
index=0;
error=30;

while [[ $index -lt $(echo "${#unique_plus_peaks[@]}")-1 ]];
do
    temp=$(( ${unique_plus_peaks[$index + 1]} - ${unique_plus_peaks[$index]} ))
    if [ $(echo "$temp") -lt $error ]
    then
        temp2=${unique_plus_peaks[$index]}
        unique_plus_peaks=($(echo ${unique_plus_peaks[@]/$temp2}))
    else
        ((index=index+1))
    fi
done

index=0;

while [[ $index -lt $(echo "${#unique_minus_peaks[@]}")-1 ]];
do
    temp=$(( ${unique_minus_peaks[$index + 1]} - ${unique_minus_peaks[$index]} ))
    if [ $(echo "$temp") -lt $error ]
    then
        temp2=${unique_minus_peaks[$index+1]}
        unique_minus_peaks=($(echo ${unique_minus_peaks[@]/$temp2}))
    else
        ((index=index+1))
    fi
done

In [3]:
echo "${unique_plus_peaks[@]}"
echo "${unique_minus_peaks[@]}"

287 465 948 1027 1075 1135 1215 1263 1534 1689 1817 1924 2659 2701 3772 3896 3969 4099 4143 4229 4314 4353 4403 4457 4540 4660 4930 5618 6946 8301 8441 8534 8823 9763 9835 9892 9964 10013 10121 10232 10364 10428 10511 10586 10884 11226 11483 12741 15075 17367 18318 18920 19946 20184 21054 21202 26 22088 26318 26818 26873 26996 27856 28945 35304 35445 35585 37311 39374 39472 39541 39727 39762 39824 39868 39917 40028 40138 40224 40411 41025 41463 42229 42741 42914 43045 43125 43626 43804 44886 47910 59974 62803 65446 68888 68965
3349 9582 11434 12605 12669 14160 16050 16250 16822 24458 24575 25033 27816 27865 28225 28274 28485 28642 29020 29070 29274 29496 33575 34352 34476 34559 34783 34836 35111 36621 37020 38461 38674 40956 42201 52965 53145 53202 53275 53351 53418 53506 53580 53736 53777 53815 53940 55501 56088 57663 57946 58075 58262 59108 59943 60374 60503 60539 60620 61020 61061 61162 61333 61896 62196 62306 62438 62590 62796 63512 63610 63714 63816 63868 64227 64365 64567 64605 6

In [4]:
mkdir genomes
awk -v OFS='\t' {'print $1,$2'} fastaFiles/LUZ7.fasta.fai > genomes/LUZ7.genome

mkdir: cannot create directory ‘genomes’: File exists


In [8]:
mkdir coverageDrops
mkdir coverageDrops/plus
mkdir coverageDrops/minus
for file in input_bam_files/BAM_files/*_enriched-LUZ7.sorted.bam
do
    bedFile=$(bedtools bamtobed -i $file | sort -k1,1 -k2,2n | grep "NC_013691.1")
    posBedFile=$(grep -w "+" <(echo "$bedFile"))
    negBedFile=$(grep -w "-" <(echo "$bedFile"))
    
    fileName=${file##*/}
    fileName="${fileName%%.*}"
    echo "$file"
    > coverageDrops/plus/${fileName}.plus.coverage.drop
    > coverageDrops/plus/${fileName}.minus.coverage.drop
    for peak in ${unique_plus_peaks[@]}
    do
        reducedBedFile=$(awk -v TTS="$peak" '$2 < TTS-10' <(echo "$posBedFile"))
        genomeCovFile=$(bedtools genomecov -g genomes/LUZ7.genome -i <(echo "$reducedBedFile") -d)
        upstreamTTSaverageCoverage=$(awk -v TTS="$peak" '$2 < TTS && $2 >= TTS-20' <(echo "$genomeCovFile") | \
        awk -v TTS="$peak" '{total += $3} END {print TTS, TTS+20, total/NR}')
        downstreamTTSaverageCoverage=$(awk -v TTS="$peak" '$2 > TTS && $2 <= TTS+20' <(echo "$genomeCovFile") |\
        awk -v TTS="$peak" '{total += $3} END {print TTS, TTS+20, total/NR}')
        upstreamValue=$(awk '{ print $3 }' <(echo "$upstreamTTSaverageCoverage"))
        if (( $(echo $upstreamValue'>'0 | bc -l) ));
        then
            coverageDrop=$(paste -d " " <(echo "$upstreamTTSaverageCoverage") <(echo "$downstreamTTSaverageCoverage") |\
            awk '{print $1, $2, $3, $4, $5, $6, $6/$3, 1-$6/$3}')
            echo "$coverageDrop" >> coverageDrops/plus/$fileName.plus.coverage.drop
        fi
    done
    
    for peak in ${unique_minus_peaks[@]}
    do
        reducedBedFile=$(awk -v TTS="$peak" '$2 < TTS-10' <(echo "$posBedFile"))
        genomeCovFile=$(bedtools genomecov -g genomes/LUZ7.genome -i <(echo "$reducedBedFile") -d)
        downstreamTTSaverageCoverage=$(awk -v TTS="$peak" '$2 < TTS && $2 >= TTS-20' <(echo "$genomeCovFile") | \
        awk -v TTS="$peak" '{total += $3} END {print TTS, TTS+20, total/NR}')
        upstreamTTSaverageCoverage=$(awk -v TTS="$peak" '$2 > TTS && $2 <= TTS+20' <(echo "$genomeCovFile") |\
        awk -v TTS="$peak" '{total += $3} END {print TTS, TTS+20, total/NR}')
        upstreamValue=$(awk '{ print $3 }' <(echo "$upstreamTTSaverageCoverage"))
        if (( $(echo $upstreamValue'>'0 | bc -l) ));
        then
            coverageDrop=$(paste -d " " <(echo "$upstreamTTSaverageCoverage") <(echo "$downstreamTTSaverageCoverage") |\
            awk '{print $1, $2, $3, $4, $5, $6, $6/$3, 1-$6/$3}')
            echo "$coverageDrop" >> coverageDrops/minus/$fileName.plus.coverage.drop
        fi
    done
done

mkdir: cannot create directory ‘coverageDrops’: File exists
mkdir: cannot create directory ‘coverageDrops/plus’: File exists
mkdir: cannot create directory ‘coverageDrops/minus’: File exists
input_bam_files/BAM_files/t10_enriched-LUZ7.sorted.bam
input_bam_files/BAM_files/t20_enriched-LUZ7.sorted.bam
input_bam_files/BAM_files/t5_enriched-LUZ7.sorted.bam


1


test
